In [1]:
import ee
import datetime
import os
import itertools
import sys
import urllib.request

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In C:\Users\wangj\Anaconda3\envs\ee_py3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\wangj\Anaconda3\envs\ee_py3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\wangj\Anaconda3\envs\ee_py3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\wangj\Anaconda3\envs\ee_py3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\wangj\Anaconda3\envs\ee_py3\lib\

In [2]:
ee.Initialize()

##### Define basic parameters

In [3]:
region_en = 'huadong'
region_cn = '华东'

In [4]:
grid_shp  =  ee.FeatureCollection("users/wangjinzhulala/China_built_up/01_Boundary_shp/China_zone_grid_40km")\
               .filterMetadata('NAME1','equals',region_cn)

sample_path = 'users/wangjinzhulala/China_built_up/03_sample_extract_img_value'

In [5]:
# define the gee-asset path for exporting
export_path = 'users/wangjinzhulala/China_built_up/04_sample_train_test_split'

In [6]:
# define the year range
year_start = [i for i in range(1990,2018,3)]
year_end   = [i for i in range(1992,2020,3)]

year_name = [f'{i[0]}_{i[1]}' for i in zip(year_start,year_end)]

In [7]:
year_name

['1990_1992',
 '1993_1995',
 '1996_1998',
 '1999_2001',
 '2002_2004',
 '2005_2007',
 '2008_2010',
 '2011_2013',
 '2014_2016',
 '2017_2019']

##### Using grid to extract sample points

In [8]:
# Define a spatial filter as geometries that intersect.
spatialFilter = ee.Filter.intersects(
                                      leftField= '.geo',
                                      rightField= '.geo',
                                      maxError= 1
                                    )

In [9]:
# Define a save all join.
saveAllJoin = ee.Join.saveAll(matchesKey= 'sample_pts')

In [12]:
for year in year_name:
    
    # get sample
    sample_pt = ee.FeatureCollection(f"{sample_path}/Control_sample_ext_img_{region_en}_{year}") 
    
    pt_built     = sample_pt.filterMetadata('Built','equals',1)
    pt_non_built = sample_pt.filterMetadata('Built','equals',0)
    
    
    #_____________________________1: select one point from each grid________________________
    # Apply the join.
    grid_intersect_bulit = saveAllJoin.apply(grid_shp, pt_built, spatialFilter)
  
    # select one point from each grid
    choose_one_built_pt = grid_intersect_bulit.map(lambda fe: ee.List(fe.get('sample_pts')).get(0) )
    
    # balance the size of choose_one sample [built = non-built]
    choose_one_non_built_pt = pt_non_built\
                              .randomColumn()\
                              .sort('random')\
                              .limit(choose_one_built_pt.size())

    # merge to get the hold_out sample
    hold_out = choose_one_built_pt.merge(choose_one_non_built_pt)  
    
    #_____________________________2: exclude the selected points________________________
    
    # using the invert join for exclusion
    invertedJoin = ee.Join.inverted()
    hold_in = invertedJoin.apply(sample_pt, hold_out, spatialFilter)
    
    
    #_____________________________3: download the csv to local drive & gee-asset_______________
    
    # construct export names
    name_hold_out = f'Grid_select_{region_en}_{year}'
    name_hold_in  = f'Training_sample_{region_en}_{year}' 

    # downloding
    for table_name,pts in zip([name_hold_out,name_hold_in],[hold_out,hold_in]):
        
        size = hold_out.size().getInfo()
        
        # to loacl drive
        ulr = pts.getDownloadURL('csv') 
        urllib.request.urlretrieve(ulr, f'./Data/{table_name}.csv')
        
        # to GEE-asset
        task = ee.batch.Export.table.toAsset(collection=pts,
                                             description=table_name,
                                             assetId=f'{export_path}/{table_name}')
        
        task.start()
        
    # print out the process
    print(f'{region_en}_{year} of {size} points downloaded!')

huadong_1990_1992 of 818 points downloaded!
huadong_1993_1995 of 818 points downloaded!
huadong_1996_1998 of 818 points downloaded!
huadong_1999_2001 of 818 points downloaded!
huadong_2002_2004 of 818 points downloaded!
huadong_2005_2007 of 818 points downloaded!
huadong_2008_2010 of 818 points downloaded!
huadong_2011_2013 of 818 points downloaded!
huadong_2014_2016 of 818 points downloaded!
huadong_2017_2019 of 818 points downloaded!
